In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, TensorDataset

from typing import List, Tuple, NoReturn, Any, Optional, Union
import pandas as pd
import numpy as np
import random

from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

from tqdm.auto import tqdm, trange
import os

import datasets
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, HfArgumentParser
import re

In [2]:
dataset = load_from_disk("/opt/ml/data/train_dataset")
train_dataset = dataset["train"]
valid_dataset = dataset["validation"]
dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [3]:
model_checkpoint = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [42]:
df = pd.read_csv("/opt/ml/data/bm25/bm25_top5_pp1_val.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/opt/ml/data/bm25/bm25_top5_pp1_val.csv'

In [35]:
answers = df.answers.to_list()
question = df.question.to_list()
context_id = df.context_id.to_list()
original_context = df.original_context.to_list()
top_k_passage = []
target = []

In [36]:
len(answers), len(question), len(original_context)

(240, 240, 240)

In [37]:
for idx, passages in enumerate(df.context.tolist()):
    p_list = passages.split('▦')
    
    if original_context[idx] in p_list:
        top_k_passage.append(p_list)
        target_index = p_list.index(original_context[idx])
        target.append(target_index)
    else :
        p_list = [original_context[idx]] + p_list[:-1]
        top_k_passage.append(p_list)
        target.append(0)

In [39]:
original_context[0]

'순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\\n\\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. \\n\\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.'

In [11]:
file_name = "dksahjklvjxzckl \
asdkljklzxjckljvcxzk \
ksdajklxzcvjklvxcz\
fjklasdjkldscv\
dfsjkljxckzl"

In [12]:
file_name

'dksahjklvjxzckl asdkljklzxjckljvcxzk ksdajklxzcvjklvxczfjklasdjkldscvdfsjkljxckzl'